### Weather stations and Municipal water locations

In [1]:
!cat ../../../../data/geography/weather_stations.csv | head











cat: write error: Broken pipe


In [2]:
!cat ../../../../data/geography/water_systems.csv | head











cat: write error: Broken pipe


In [7]:
weather_station_path = '../../../../data/geography/weather_stations.csv'

In [8]:
water_system_path = '../../../../data/geography/water_systems.csv'

In [3]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic #this isn't actually very useful as it's ridiculously slow.
                                    #I've instead done a rough approximate by querying a spatial.KDTree

In [9]:
water_systems = pd.read_csv(water_system_path) #the csvs are mis-named

In [10]:
len(water_systems)

146351

In [11]:
water_systems[:10]

,PWSID,LAT,LON
0,10106001,41.449482,-71.98233
1,10109005,41.457198,-72.11459
2,10307001,41.345578,-70.75145
3,10502002,41.385256,-71.66813
4,10502003,41.385256,-71.66813
5,20000001,42.168508,-78.72970
6,20000004,42.168508,-78.72970
7,20000005,44.981840,-74.67374
8,20000007,42.168508,-78.72970
9,20000008,42.168508,-78.72970


In [12]:
#weather_stations = pd.read_csv('weatherStationLocation.csv')
weather_stations = pd.read_csv(weather_station_path)

In [13]:
weather_stations[:10]

,Station,Lat,Lon
0,STOVEPIPE WELLS 1 SW,36.60200,-117.14490
1,DEATH VALLEY,36.46220,-116.86690
2,IMPERIAL COUNTY AP,32.83417,-115.57861
3,YUMA 13.8 ESE,32.63410,-114.40560
4,YUMA 7.7 SE,32.64320,-114.52300
5,YUMA PROVING GROUND,32.83560,-114.39420
6,YUMA 27 ENE,32.83500,-114.18840
7,SOMERTON 3.0 ESE,32.56760,-114.65550
8,INDIO FIRE STATION,33.70860,-116.21520
9,TWENTYNINE PALMS,34.12800,-116.03690


In [14]:
weather_stations.dtypes

Station     object
Lat        float64
Lon        float64
dtype: object

In [15]:
water_systems.dtypes

PWSID     object
LAT      float64
LON      float64
dtype: object

In [16]:
len(weather_stations)

9191

In [17]:
len(water_systems)

146351

In [18]:
lat_weather, lon_weather = weather_stations['Lat'][0], weather_stations['Lon'][0]

In [19]:
print(lat_weather, lon_weather)

36.602 -117.1449


In [20]:
lat_water, lon_water = water_systems['LAT'][0], water_systems['LON'][0]

In [21]:
print(lat_water, lon_water)

41.449482 -71.98233


In [22]:
weather_coord = lat_weather, lon_weather

In [23]:
water_coord = lat_water, lon_water

In [24]:
weather_coord

(36.602, -117.1449)

In [25]:
water_coord

(41.449482, -71.98233)

### Test data:

In [26]:
pd_tmp = pd.DataFrame(40 * np.random.random_sample(size=(100, 2)) + 30) #random coordinates to play with

In [27]:
pd_tmp_2 = pd.DataFrame(40 * np.random.random_sample(size=(100, 2)) + 30) #random coordinates to play with

In [28]:
pd_tmp.columns = ['Lat', 'Lon']

In [29]:
pd_tmp_2.columns = ['Lat', 'Lon']

In [30]:
pd_tmp[:10]

,Lat,Lon
0,50.029622,31.184756
1,67.170417,42.855607
2,63.574464,55.309269
3,68.680506,49.630252
4,47.833108,57.202716
5,64.228282,57.245945
6,48.971014,48.712493
7,64.505328,45.773187
8,69.038500,51.211282
9,39.936606,47.946563


In [31]:
pd_tmp['dist'] = pd_tmp.apply(lambda row: geodesic((row.Lat, row.Lon), water_coord).miles, axis=1)

In [32]:
pd_tmp[:10]

,Lat,Lon,dist
0,50.029622,31.184756,4612.176333
1,67.170417,42.855607,4212.308524
2,63.574464,55.309269,4642.671380
3,68.680506,49.630252,4276.306762
4,47.833108,57.202716,5545.938591
5,64.228282,57.245945,4645.580432
6,48.971014,48.712493,5239.609031
7,64.505328,45.773187,4395.198033
8,69.038500,51.211282,4287.418069
9,39.936606,47.946563,5683.528140


Returns the index of the smallest distance:

In [33]:
pd_tmp.apply(lambda inner_row: geodesic((inner_row.Lat, inner_row.Lon), water_coord).miles, axis=1).idxmin()

24

In [34]:
pd_tmp_2[:3]

,Lat,Lon
0,68.583610,49.784448
1,45.142953,53.580836
2,54.531124,64.946319


### Approximate distances computed with a KDTree:

In [35]:
#stackoverflow says use a KDTree
#https://stackoverflow.com/questions/10818546/finding-index-of-nearest-point-in-numpy-arrays-of-x-and-y-coordinates

In [36]:
from scipy import spatial #this is where KDTree imports from

In [37]:
water_systems[['LAT', 'LON']].values #to initialize the KDTree, we need an array.
                                     #We use this format below.

array([[  41.449482,  -71.98233 ],
       [  41.457198,  -72.11459 ],
       [  41.345578,  -70.75145 ],
       ...,
       [  42.729832, -108.63154 ],
       [  42.729832, -108.63154 ],
       [  44.801249, -106.96782 ]])

In [38]:
len(weather_stations), len(water_systems) #seeing again how big each df is.

(9191, 146351)

In [40]:
%%time
#initializing the KDTree on weather_stations.
#We then poll this object in the next step over each line in water_systems.
weather_kd_tree = spatial.KDTree(weather_stations[['Lat', 'Lon']].values)

CPU times: user 88 ms, sys: 0 ns, total: 88 ms
Wall time: 92.5 ms


In [41]:
#The following cell takes about 1 minute 15 seconds to run on a 2010 era middling laptop:

In [42]:
%%time
water_weather_indexes = water_systems.apply(lambda row: np.asarray(weather_kd_tree.query([row.LAT, row.LON])), axis=1)

CPU times: user 1min 14s, sys: 176 ms, total: 1min 14s
Wall time: 1min 14s


In [44]:
len(water_weather_indexes)

146351

The returned distance is measured as if the coordinates were points on a flat grid. 
The units are a messy tangle of Lat and Lon but this seems good enough for our purposes.

In [45]:
water_weather_indexes[:10] #the format is a strangely united distance (it's a rough approximate)
                           #followed by the index where that coordinate lives in weather_stations

0    [0.11267760923980558, 5448.0]
1    [0.08600884201057854, 5448.0]
2    [0.20866895069463523, 5830.0]
3    [0.13550271176621798, 5517.0]
4    [0.13550271176621798, 5517.0]
5    [0.07107975284144687, 7837.0]
6    [0.07107975284144687, 7837.0]
7    [0.17813446662563626, 3313.0]
8    [0.07107975284144687, 7837.0]
9    [0.07107975284144687, 7837.0]
dtype: object

In [46]:
#splitting the array into two separate series:
distances, station_indexes = (np.vstack(water_weather_indexes)[:,0],
                              np.vstack(water_weather_indexes)[:,1])

In [47]:
distances

array([0.11267761, 0.08600884, 0.20866895, ..., 0.06585048, 0.06585048,
       0.01385044])

In [48]:
station_indexes

array([5448., 5448., 5830., ..., 2170., 2170., 1746.])

In [49]:
max(station_indexes)

9187.0

In [50]:
len(station_indexes)

146351

In [51]:
station_indexes_df = pd.DataFrame(station_indexes) #converting to a pandas dataframe

In [52]:
station_indexes_df[:10]

,0
0,5448.0
1,5448.0
2,5830.0
3,5517.0
4,5517.0
5,7837.0
6,7837.0
7,3313.0
8,7837.0
9,7837.0


In [53]:
test = pd.DataFrame(weather_stations[:5]) #making sure we're looking at the right thing

In [54]:
test

,Station,Lat,Lon
0,STOVEPIPE WELLS 1 SW,36.60200,-117.14490
1,DEATH VALLEY,36.46220,-116.86690
2,IMPERIAL COUNTY AP,32.83417,-115.57861
3,YUMA 13.8 ESE,32.63410,-114.40560
4,YUMA 7.7 SE,32.64320,-114.52300


In [55]:
water_systems.iloc[1000] #getting the value based on an int index

PWSID    AK2213598
LAT        61.0683
LON         -149.8
Name: 1000, dtype: object

In [56]:
len(water_systems)

146351

In [57]:
water_systems.iloc[9190]

PWSID    CA3301145
LAT        33.7508
LON       -116.726
Name: 9190, dtype: object

In [58]:
station_indexes[:100]

array([5448., 5448., 5830., 5517., 5517., 7837., 7837., 3313., 7837.,
       7837., 6318., 3106., 7837., 7837., 8001., 7159., 7159., 7159.,
       7159., 8700., 8700., 8079., 8079., 8079., 8079., 6771., 6771.,
       6771., 6771., 6771., 6771., 6771., 6771., 6771., 6771., 6771.,
       6771., 6771., 6771., 6771., 6516., 6771., 6771., 6771., 4558.,
       3714., 4753., 4253., 4253., 4253., 6168., 6168., 6168., 4478.,
       4478., 4478., 5150., 6936., 2783., 2783., 2783., 2783., 2783.,
       6563., 6563., 1632., 8422., 6436., 6436., 6436., 6436., 6217.,
       3913., 3913., 3913., 3913., 4373., 4558., 3714., 4753., 4753.,
       5403., 4253., 4253., 4253., 3146., 3146., 2852., 3680., 4478.,
       4478., 4904., 5353., 2783., 2783., 2783., 2783., 2783., 2783.,
       2783.])

In [59]:
len(station_indexes), len(water_systems), len(weather_stations)

(146351, 146351, 9191)

The following cell takes about a minute to run on my 2010 thinkpad:

In [60]:
%%time
#look up each index in water_systems and assign it to a row.
#paired_stations = station_indexes_df.apply(lambda row: water_systems.iloc[int(row)], axis=1)
paired_stations = station_indexes_df.apply(lambda row: weather_stations.iloc[int(row)], axis=1)

CPU times: user 1min 1s, sys: 212 ms, total: 1min 1s
Wall time: 1min 2s


In [113]:
#paired_stations.columns = ['weather_station_ID', 'station_lat', 'station_lon']

In [62]:
paired_stations[:10]

,Station,Lat,Lon
0,NORWICH PUB UTILITY PLANT,41.52690,-72.06420
1,NORWICH PUB UTILITY PLANT,41.52690,-72.06420
2,WOODS HOLE GOLF CLUB,41.53260,-70.65890
3,WESTERLY STATE AIRPORT,41.34972,-71.79889
4,WESTERLY STATE AIRPORT,41.34972,-71.79889
5,ALLEGANY STATE PARK,42.10030,-78.74970
6,ALLEGANY STATE PARK,42.10030,-78.74970
7,MASSENA INTL AP,44.93583,-74.84583
8,ALLEGANY STATE PARK,42.10030,-78.74970
9,ALLEGANY STATE PARK,42.10030,-78.74970


In [63]:
%%time
stations_and_systems = pd.concat([water_systems, paired_stations], axis=1)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 8.4 ms


In [64]:
stations_and_systems[:5] #making sure everything is in the right format

,PWSID,LAT,LON,Station,Lat,Lon
0,10106001,41.449482,-71.98233,NORWICH PUB UTILITY PLANT,41.52690,-72.06420
1,10109005,41.457198,-72.11459,NORWICH PUB UTILITY PLANT,41.52690,-72.06420
2,10307001,41.345578,-70.75145,WOODS HOLE GOLF CLUB,41.53260,-70.65890
3,10502002,41.385256,-71.66813,WESTERLY STATE AIRPORT,41.34972,-71.79889
4,10502003,41.385256,-71.66813,WESTERLY STATE AIRPORT,41.34972,-71.79889


In [65]:
stations_and_systems.rename(index=str, columns={"Station": "weather_station", 
                                                "Lat": "weather_lat", 
                                                "Lon":"weather_lon"})

,PWSID,LAT,LON,weather_station,weather_lat,weather_lon
0,10106001,41.449482,-71.98233,NORWICH PUB UTILITY PLANT,41.52690,-72.06420
1,10109005,41.457198,-72.11459,NORWICH PUB UTILITY PLANT,41.52690,-72.06420
2,10307001,41.345578,-70.75145,WOODS HOLE GOLF CLUB,41.53260,-70.65890
3,10502002,41.385256,-71.66813,WESTERLY STATE AIRPORT,41.34972,-71.79889
4,10502003,41.385256,-71.66813,WESTERLY STATE AIRPORT,41.34972,-71.79889
5,20000001,42.168508,-78.72970,ALLEGANY STATE PARK,42.10030,-78.74970
6,20000004,42.168508,-78.72970,ALLEGANY STATE PARK,42.10030,-78.74970
7,20000005,44.981840,-74.67374,MASSENA INTL AP,44.93583,-74.84583
8,20000007,42.168508,-78.72970,ALLEGANY STATE PARK,42.10030,-78.74970
9,20000008,42.168508,-78.72970,ALLEGANY STATE PARK,42.10030,-78.74970


In [121]:
!pwd

/home/aaron/projects/notebooks/water project


In [66]:
csv_output_path = '../../../../data/geography/water_systems_and_nearest_weather_stations.csv'

In [68]:
stations_and_systems.to_csv(csv_output_path)